## Coordinates

In acoustics research, we often deal with different coordinate systems when handling sampling points. This can get complicated. That's why we have the Coordinates class in pyfar. The :py:func:`Coordinates class <pyfar.classes.coordinates.Coordinates>` was designed for storing, working with, and accessing coordinate points. You can easily switch between different coordinate systems, rotate points, and do other useful stuff. For example, you can store microphone positions in a spherical microphone array or loudspeaker positions in a sound field synthesis system.

In [ ]:
import pyfar as pf
import numpy as np
import matplotlib.pyplot as plt



## Supported Coordinate systems

Coordinate systems come in different flavors, like cartesian, spherical or cylindrical, where spherical has a couple of subdefinitions. The image below shows the available coordinate systems.
![alt text](../../resources/coordinate_systems.png)


### Entering coordinate points

You can input coordinate points manually into the system. By default, the system assumes Cartesian coordinates. Internally, all points are consistently stored in the Cartesian coordinate system.

In [ ]:
c = pf.Coordinates(np.arange(-5., 6), 0, 0)
# plot the sampling points
c.show()

To initialize `Coordinates` objects for different coordinate systems, you can use specific constructors like `pf.Coordinates.from_spherical_elevation(azimuth, elevation, radius)`. The naming convention for these constructors is always `pf.Coordinates.from_coordinate_system(red, green, blue)`, where `coordinate_system` should be replaced with the desired coordinate system, and `red`, `green`, `blue` describe the order of the coordinate properties. Remember, angles are always defined in radians.

For more details, you can refer to the [coordinate class documentation](https://pyfar.readthedocs.io/en/latest/classes/pyfar.coordinates.html).

In [ ]:
azimuth_angles = np.arange(0, 2*np.pi, np.pi/20)
c1 = pf.Coordinates.from_spherical_elevation(azimuth_angles, 0, 1)
c1.show()
plt.show()

### Meta data

Lets have a look on the other data. At first we can print the coordinate object.


In [ ]:
print(c1)

We can observe that there are 40 points in total over all dimensions. This refers to the attribute `csize`.


In [ ]:
c1.csize

The cshape of the data is (40,). The term cshape refers to the shape of a single coordinate.

In [ ]:
c1.cshape

Similarly, cdim returns the number of dimensions of the data.

In [ ]:
c1.cdim

The `cshape`, `csize`, and `cdim` attributes are similar to numpy's `shape`, `size`, and `dim` of each coordinate.


### Retrieving coordinate points

There are different ways to retrieve points from a `Coordinates` object. All points can be obtained in cartesian, spherical, and cylindrical coordinates using the related properties `c.cartesian`, `c.sperical_evaluation` and `c.cylindrical`. Also single properties of each coordinate system convention can be accessed by e.g. `c.azimuth`, `c.radius` or `c.x`. Visit the [coordinate class](https://pyfar.readthedocs.io/en/latest/classes/pyfar.coordinates.html) for more details."

In [ ]:
cartesian_coordinates = c.cartesian
cartesian_coordinates

Of course, we can convert it into other coordinate systems as well. Note that it returns angles always in radiance.

In [ ]:
c.spherical_elevation

We can also directly access certain coordinates.

In [ ]:
c.azimuth

### Manipulating points
the previous attributes can also be used to manipualte the data.

In [ ]:
c.azimuth[0] = 0
c.azimuth

### Find a specific subset
Different methods are available for obtaining a specific subset of coordinates.


#### Find nearest
The first method is to find the nearest points in a subset. We need to define one or more coordinates which we want to find.

In [ ]:
find = pf.Coordinates.from_spherical_colatitude(270/180*np.pi, 90/180*np.pi, 1)
index_out, distance = c.find_nearest(find)
c.show(index_out)
plt.show()

If we want to find the nearest 3 points, then we need to set `k=3`. In this example we highlight the second nearest point.

In [ ]:
index_out, distance = c.find_nearest(find, k=3)
c.show(index_out[1])
plt.show()
distance


#### Find within
Another option is to find all points in a certain area around different points. Different distance measures are available see the [pyfar documentation](https://pyfar.readthedocs.io/en/latest/classes/pyfar.coordinates.html#pyfar.classes.coordinates.Coordinates.find_within) for more information.

In [ ]:
index_out = c.find_within(find, distance=3, distance_measure='euclidean')
c.show(index_out)
plt.show()

Another way is to use logical operations on the coordinates attributed directly.
To obtain all points within a specified euclidean distance or arc distance, you can use `c.get_nearest_cart()` and `c.get_nearest_sph()`. To obtain more complicated subsets of any coordinate, e.g., the horizontal plane with `colatitude=90` degree, you can use

In [ ]:
index_out = c.colatitude == 90/180*np.pi
c.show(index_out)
plt.show()

### Rotating coordinates

You can apply rotations using quaternions, rotation vectors/matrixes and euler angles with  `c.rotate()`, which is a wrapper for `scipy.spatial.transform.Rotation`. For example rotating around the y-axis by 45 degrees can be done with

In [ ]:
c.rotate('y', 45)
c.show()
plt.show()

Note that this changes the points inside the `Coordinates` object, which means that you have to be careful not to apply the rotation multiple times, i.e., when evaluationg cells during debugging.

## Orientations

The `Orientations()` class is designed for storing, manipulating, and accessing orientation vectors. Examples for this are orientations of directional loudspeakers during measurements or head orientations. It is good to know that `Orientations` is inherited from `scipy.spatial.transform.Rotation` and that all methods of this class can also be used with `Orientations`.

### Entering orientations

Lets go ahead and create an object and show the result

In [ ]:
views = [[0,  1, 0],
         [1,  0, 0],
         [0, -1, 0]]
up = [0, 0, 1]
orientations = pf.Orientations.from_view_up(views, up)
orientations.show(show_rights=False)


It is also possible to enter `Orientations` from `Coordinates` object or mixtures of `Coordinates` objects and array likes. This is equivalent to the example above

In [ ]:
azimuths = np.array([90, 0, 270]) * np.pi / 180
views_c = pf.Coordinates.from_spherical_elevation(azimuths, 0, 1)

orientations = pf.Orientations.from_view_up(views_c, up)

### Retrieving orientations

Orientaions can be retrieved as view, up, and right-vectors and in any format supported by `scipy.spatial.transform.Rotation`. They can also be converted into any coordinate convention supported by pyfar by putting them into a `Coordinates` object. Lets only check out one way for now 

In [ ]:
views, ups, right, = orientations.as_view_up_right()
views

### Rotating orientations

Rotations can be done using the methods inherited from `scipy.spatial.transform.Rotation`. You can for example rotate around the y-axis this way

In [ ]:
rotation = pf.Orientations.from_euler('y', 30, degrees=True)
orientations_rot = orientations * rotation
orientations_rot.show(show_rights=False)